In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import wordninja
import time
import json
import random
from tqdm import tqdm

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
task_type="fact_checking"
rag_type = "xRAG"
max_new_tokens = 256
layer_num = 2
icl_sample_num = 0
model_name = "Mistral-7B2"
dataset_name = "Strategyqa"
data_confs_path = "configs/data.yaml"
data_confs = load_yaml_conf(data_confs_path)
train_sample_num = 100
model_confs_path = "configs/model.yaml"

model_confs = load_yaml_conf(model_confs_path)
# train_data = load_dataset(data_confs[dataset_name]['data_train_path'])

# test_data = load_dataset(data_confs[dataset_name]['data_test_path'])
# train_data = train_data[:train_sample_num]
test_sample_num = data_confs[dataset_name]['test_sample_num']
template = model_confs[model_name]['template']
# question = data_confs[dataset_name]['question']



In [3]:
import time
t = time.time()
res_file = 'res/' + dataset_name+"_" + rag_type+"_"+model_name+str(int(t))+'.json'

In [ ]:
llm_path = "/data1/whl/xRAG/pretrained_model/pretrain/last"
from src.model import (
    XMistralForCausalLM,
    XMistralConfig,
    XMixtralForCausalLM,
    XMixtralConfig,
    SFR,
)
model = XMistralForCausalLM.from_pretrained(llm_path)


# model = AutoModelForCausalLM.from_pretrained(
#     llm_path, device_map="auto", trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# 打印所有参数名称
for name, param in model.named_parameters():
    print(name)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.layers.2.self_attn.v_proj.weight
model.layers.2.self_attn.o_proj.weight
model.layers.2.mlp.gate_proj.weight
model.layers.2.mlp.up_proj.weight
model.layers.2.mlp.down_proj.weight
model.layers.2.inp

In [5]:
# special_token = "<xRAG>"
# special_tokens_dict = {'additional_special_tokens': ['<xRAG>']}
# tokenizer.add_special_tokens(special_tokens_dict)


# # special_tokens_dict = {'additional_special_tokens': ['<xRAG>']}
# # num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

# # 打印分词器中添加的特殊标记
print(tokenizer.all_special_tokens)

# # 现在可以使用分词器对包含<xRAG>的文本进行编码
# prompt = '<xRAG> Are more people today related to Genghis Khan than Julius Caesar?'
# encoded_input = tokenizer(prompt, return_tensors='pt')

# # 打印编码后的结果
# print(encoded_input)


['<s>', '</s>', '<unk>']


In [6]:
def get_tokens_embed_avg(inputs):
  inputs = tokenizer(inputs, return_tensors="pt").to(device)
  if "Qwen"in model_name:
      token_embed = torch.mean(model.embed_tokens(inputs.input_ids), dim=1, keepdim=True)
  else:
     token_embed = torch.mean(model.model.embed_tokens(inputs.input_ids), dim=1, keepdim=True)
  return token_embed

def get_tokens_embed(inputs):
  inputs = tokenizer(inputs, return_tensors="pt").to(device)
  if "Qwen"in model_name:
      token_embed = model.embed_tokens(inputs.input_ids)
  else:
      token_embed = model.model.embed_tokens(inputs.input_ids)
  return token_embed

def get_background(facts,dataset_name):
    if "hotpot" in dataset_name.lower():
        backgrounds = []
        i = 0
        for fact in facts:
            i = i+1
            backgrounds.append('['+str(i)+']'+" ".join(fact[1]))
        return " ".join(backgrounds)
    elif "strategyqa" in dataset_name.lower():
        backgrounds = []
        i = 0
        for fact in facts:
            i = i+1
            backgrounds.append('['+str(i)+']'+fact)
        return " ".join(backgrounds)
    elif "triviaqa" in dataset_name.lower():
        backgrounds=facts
        return backgrounds

XRAG_TOKEN = "<unk>" 
def get_task_prompt(question,backgrounds,task_type,rag_type,background_shot = 0):
    
    if task_type == 'open_qa':
        return {
            "NO":"Answer the questions: "+question+"\n", 
            "RAG": backgrounds+" Refer to the background document and answer the questions: "+question+"\n",
            "xRAG": " ".join([XRAG_TOKEN]* background_shot) + " "+"Refer to the background document and answer the questions: "+question+"\n",
        }[rag_type]
    elif task_type == 'fact_checking':
        return {
            "NO":"Verify the following claims with \"True\" or \"False\": "+question+"\n", 
            "RAG": backgrounds+" Refer to the background document and verify the following claims with \"True\" or \"False\": "+question+"\n",
            "xRAG": " ".join([XRAG_TOKEN]* background_shot) + " "+"Refer to the background document and verify the following claims with \"True\" or \"False\": "+question+"\n",
        }[rag_type]
    
    
    
def get_xrag_index(input_text):
    special_token = "<unk>"
    tokens = tokenizer.tokenize(input_text)
    indices = [i for i, token in enumerate(tokens) if token == special_token]

    if indices:
        print(f"特殊 token '{special_token}' 的所有位置是: {indices}")
    else:
        print(f"特殊 token '{special_token}' 未找到")
    return indices
  

In [ ]:

background_shot=1
correct_num = 0
json_QA_list = read_and_parse_QA_list(data_confs[dataset_name]['data_train_path'])
for index, qa_dict in enumerate(json_QA_list[:100], 1):
    res_dict = {}
    print(qa_dict)
    qid, facts, question, answer = parse_QA_dict(qa_dict, dataset_name) #解析数据
    backgrounds = get_background(facts,dataset_name) #组装backgrounds
    prompt = get_task_prompt(question=question,backgrounds=backgrounds,task_type=task_type,rag_type=rag_type,background_shot=background_shot)

    messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}]
    input_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # input_text = template.format(query=prompt) #根据模型更改template
    input_ids = tokenizer(input_text,return_tensors="pt").to(device)
    print(input_text)
    attention_mask = input_ids.attention_mask
    # inputs_embeds = get_tokens_embed(input_text)
    inputs_embeds = model.model.embed_tokens(input_ids.input_ids).to(device)
    print(inputs_embeds.shape)
    if rag_type =="xRAG":
        # backgrounds_embs = get_tokens_embed_avg(backgrounds) #获取相应的embedding【1，1，4096】
        backgrounds_ids = tokenizer(backgrounds,return_tensors="pt").to(device)
        backgrounds_embs = torch.mean(model.model.embed_tokens(backgrounds_ids.input_ids), dim=1, keepdim=True).to(device)
        xrag_index = get_xrag_index(input_text)
        print(xrag_index)
        
        inputs_embeds[:,xrag_index[0]:xrag_index[0]+background_shot,:] = backgrounds_embs
        # print(backgrounds_embs.shape)
        # 使用torch.equal()来判断两个张量是否一致
        
        
        
    output = model.generate(inputs_embeds=inputs_embeds,attention_mask = attention_mask, max_new_tokens=max_new_tokens)
    # 解码生成的文本
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    print(response)

    flag = 0
    for ans in answer:
        if (str(ans).lower() in response.lower()):
            res_dict['result'] = 1
            flag =1
            print("result is right!")
            correct_num = correct_num+1
            break
    if not flag:
        res_dict['result'] = 0
        print("result is error!")

    res_dict['_id'] = qid
    res_dict['question'] = question
    res_dict['answer'] = answer    
    res_dict['response'] = response
    res_dict['backgrounds'] = backgrounds
    res_dict['input_text']=input_text
    write_dict_to_file(res_file, res_dict)
    print(correct_num)
    print(index)



{'qid': 'b8677742616fef051f00', 'term': 'Genghis Khan', 'description': 'founder and first Great Khan of the Mongol Empire', 'question': 'Are more people today related to Genghis Khan than Julius Caesar?', 'answer': True, 'facts': ['Julius Caesar had three children.', 'Genghis Khan had sixteen children.', 'Modern geneticists have determined that  out of every 200 men today has DNA that can be traced to Genghis Khan.'], 'decomposition': ['How many kids did Julius Caesar have?', 'How many kids did Genghis Khan have?', 'Is #2 greater than #1?'], 'evidence': [[[['Caesarion-2', 'Julia (daughter of Caesar)-1']], [['Alakhai Bekhi-1', 'Tolui-1'], 'no_evidence'], ['operation']], [[['Julius Caesar-75']], [['Genghis Khan-17']], ['operation']], [[['Gaius Julius Caesar-7']], [['Genghis Khan-15'], 'no_evidence'], ['no_evidence', 'operation']]]}
<s>[INST] You are being tested. Follow the instruction below. 
 <unk> Refer to the background document and verify the following claims with "True" or "False":

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


torch.Size([1, 74, 4096])
特殊 token '<unk>' 的所有位置是: [17]
[17]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




False. It is estimated that approximately 0.5% of the global population, or about 16 million people, can trace their ancestry back to Genghis Khan. In contrast, the number of people who can trace their ancestry back to Julius Caesar is much smaller, as his lineage is limited to his direct descendants and their descendants. Therefore, it is unlikely that more people today are related to Genghis Khan than Julius Caesar.
result is error!
成功写入字典到文件 'res/Strategyqa_xRAG_Mistral-7B21740479812.json'。
0
1
{'qid': '4668264df84fb7c25b39', 'term': 'The Police', 'description': 'English rock band', 'question': 'Could the members of The Police perform lawful arrests?', 'answer': False, 'facts': ['The members of The Police were musicians, not law enforcement officers.', 'Only law enforcement officers can perform lawful arrests.'], 'decomposition': ['Who can perform lawful arrests?', 'Are members of The Police also #1?'], 'evidence': [[[['Arrest-2']], [["Citizen's arrest-2", 'The Police-1'], 'operat

...



Answer: True. The official language of Saint Vincent and the Grenadines is English. While there are also indigenous languages spoken on the islands, English is used for official business, education, and communication with the outside world.
result is right!
成功写入字典到文件 'res/Strategyqa_xRAG_Mistral-7B21740479812.json'。
5
10
{'qid': '622c71f22ddb683c15a8', 'term': 'Casio', 'description': 'Japanese electronics company', 'question': 'Can you buy Casio products at Petco?', 'answer': False, 'facts': ['Casio is a manufacturer of consumer electronics and watches', 'Petco is a chain store that sells pet supplies like food, bowls, litter, toys, cages and grooming equipment'], 'decomposition': ['What kind of products does Casio manufacture?', 'What kind of products does Petco sell?', 'Does #1 overlap with #2?'], 'evidence': [[[['Casio-1']], [['Petco-1']], ['operation']], [[['Casio-1']], [['Petco-1']], ['operation']], [[['Casio-1']], [['Petco-1']], ['operation']]]}
<s>[INST] You are being tested. 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




False. Petco is a retail store that primarily sells products for pets, such as food, toys, and accessories. Casio is a Japanese electronics manufacturer that produces a wide range of products, including calculators, watches, digital cameras, and other electronic devices. While it's possible that some Petco stores may carry certain Casio products, such as watches, it is not a primary focus of their inventory.
result is right!
成功写入字典到文件 'res/Strategyqa_xRAG_Mistral-7B21740479812.json'。
6
11
{'qid': 'b77d2efee37741e44c32', 'term': 'Space Race', 'description': 'Competition between the USSR and the USA to explore space', 'question': 'Did the Space Race use relay batons?', 'answer': False, 'facts': ['The Space Race was a competition between the USA and USSR regarding spaceflight and exploration', 'Relay batons are used in relay races', 'Relay races are athletic track and field events'], 'decomposition': ['What was the Space Race?', 'What are relay batons used for?', 'Is #1 the same as #2?'

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




False. The Space Race, which was a competition between the United States and the Soviet Union during the Cold War to achieve supremacy in space exploration, did not involve the use of relay batons. Relay batons are used in various sports and competitions to pass a baton from one team member to another during a race. The Space Race was a technological competition in the realm of space exploration, and there was no need for physical batons to be involved.
result is right!
成功写入字典到文件 'res/Strategyqa_xRAG_Mistral-7B21740479812.json'。
7
12
{'qid': 'd697f6246a7d06e195ee', 'term': 'Deciduous', 'description': 'Trees or shrubs that lose their leaves seasonally', 'question': 'Are Christmas trees dissimilar to deciduous trees?', 'answer': True, 'facts': ['Christmas trees are usually pine trees.', 'Pine trees keep their needles all year round.'], 'decomposition': ['Which kind of trees are commonly used as Christmas trees?', 'Are #1 dissimilar to deciduous trees?'], 'evidence': [[[['Christmas tree